In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:
df = pd.read_json("rawfilteredbydate.json")

In [3]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [4]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [5]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [193]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle","President and CEO","Directors","Clockwise","Doubles","BFAnyc","Standing",
        "Susan Magazine","Trustees","Chicago","Yugoslavia","Steering Committee",
        "I Have A Dream"]

In [194]:
prereplace = ["friend","friends"]

In [195]:
def replaceWord(s):
    for i in range(len(prereplace)):
        s = s.replace(prereplace[i],"")
    return s

In [196]:
captions = map(lambda x:replaceWord(x),df.caption)

In [197]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [198]:
import re

In [221]:
G=nx.MultiGraph()
for caption in captions:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5 and len(name_temp[0].split(" ")) > 1 and len(name_temp[0].split(" ")) <=4:
            
            if name_temp[0] == "PatrickMcMullan" or name_temp[0] == "Patrick McMullan":
                pass
            elif name_temp[0] == "AnnieWatt":
                name.append("Annie Watt")
            elif "Mayor" in name_temp[0]:
                temp = name_temp[0].replace("Mayor ","")
                name.append(temp)
            elif len(name_temp[0].split(" and ")) == 2:
                name1 = name_temp[0].split(" and ")[0]
                name2 = name_temp[0].split(" and ")[1]
                if len(name1.split(" ")) == 1 and len(name2.split(" ")) == 2:
                    name.append(name1+ " " + name2.split(" ")[1])
                    name.append(name2)
            else:
                name.append(name_temp[0])
        for word in restr:
            if word in name:
                name.remove(word)
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [222]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [223]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [224]:
degrees = degrees.reset_index()

In [225]:
degrees[0:50]

,index,0
0,Jean Shafiroff,719
1,Gillian Miniter,569
2,Mark Gilbertson,552
3,Geoffrey Bradfield,412
4,Alexandra Lebenthal,405
5,Somers Farkas,355
6,Yaz Hernandez,348
7,Andrew Saffir,335
8,Debbie Bancroft,331
9,Eleanora Kennedy,329


In [226]:
degrees.columns = ["name","number"]

In [227]:
[i for i in degrees.name if " and " in i]

[u'Vice-Governor and Deputy Mayor']

In [228]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [229]:
import pickle

In [230]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

## Pagerank

In [209]:
pr=nx.pagerank(Gg)
pagerank=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)

In [211]:
pagerank = pagerank.reset_index()

In [212]:
pagerank.columns = ["name","pr"]

In [213]:
pagerankres = map(lambda x,y: (x,y),pagerank.name[0:100], pagerank.pr[0:100])

In [214]:
with open('pagerank.pickle', 'wb') as f:
    pickle.dump(pagerankres, f)

## Best Friend

In [215]:
wedge=[[g[0],g[1],g[2]['weight']] for g in Gg.edges(data=True)]
friends=pd.DataFrame(wedge,columns=['n1','n2','weight']).sort('weight',ascending=False)
friends.index=range(len(friends))

In [216]:
friends.head()

,n1,n2,weight
0,Gillian Miniter,Sylvester Miniter,113
1,Bonnie Comley,Stewart Lane,85
2,Jamee Gregory,Peter Gregory,73
3,Geoffrey Bradfield,Roric Tobin,71
4,Andrew Saffir,Daniel Benedict,70


In [218]:
friendsres = map(lambda n1,n2,weight: ((n1,n2),weight), friends.n1[0:100], friends.n2[0:100], friends.weight[0:100])

In [219]:
with open('friends.pickle', 'wb') as f:
    pickle.dump(friendsres, f)